In [ ]:




select tags from tb_algo where locate("机器",tags);

In [1]:
%%file Arthur_serviceTags_dao.py
from Arthur.core.entities.arthur_web_cfg import *
from Arthur.core.entities.base.bs_util import *
from Arthur.core.entities.base.bs_database_pid import DataBase
from Arthur.core.entities.base.bs_time import get_cur_day,get_cur_time
#2019-01-30 add
class ArthurServTagsDao:
    
    #用户的tag service list
    @staticmethod
    def query_node_list_by_tag(status='normal', search=None):
        dataBase = DataBase()
        sql = "select * from tb_algo where 1=1 "
        param = []
        if status and status != 'all':
            sql += "and status = %s "
            param.append(status)
        if search:
            #search = "%%%s%%" % (search)           
            sql += "and locate(%s,tags)"#tags like %s or algoname like %s
            param.append(search)

        """sql += "order by aid desc limit %s, %s"
        param.append(offset)
        param.append(limit)"""

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet

if __name__ == "__main__":

    #ArthurAlgoStarDao.insert_node(algouid, uid,algoname,star_status,thumb_status)
    ArthurServTagsDao.query_node_list_by_tag(search='机器')

Writing Arthur_serviceTags_dao.py


In [2]:
from Arthur.core.entities.arthur_web_cfg import *
from Arthur.core.entities.base.bs_util import *
from Arthur.core.entities.base.bs_database_pid import DataBase
from Arthur.core.entities.base.bs_time import get_cur_day,get_cur_time
#2019-01-30 add
class ArthurServTagsDao:
    
    #用户的tag service list
    @staticmethod
    def query_node_list_by_tag(status='normal', search=None):
        dataBase = DataBase()
        sql = "select * from tb_algo where 1=1 "
        param = []
        if status and status != 'all':
            sql += "and status = %s "
            param.append(status)
        if search:
            #search = "%%%s%%" % (search)           
            sql += "and locate(%s,tags)"#tags like %s or algoname like %s
            param.append(search)

        """sql += "order by aid desc limit %s, %s"
        param.append(offset)
        param.append(limit)"""

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet

if __name__ == "__main__":

    #ArthurAlgoStarDao.insert_node(algouid, uid,algoname,star_status,thumb_status)
    c,y = ArthurServTagsDao.query_node_list_by_tag(search='机器')

[30/01/2019 19:53:43][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_exec_cmdstr(106): select * from tb_algo where 1=1 and status = %s and locate(%s,tags)	['normal', '\xe6\x9c\xba\xe5\x99\xa8']
[30/01/2019 19:53:43][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)


In [3]:
y

[{'aid': 34L,
  'algo_tm': datetime.datetime(2019, 1, 28, 16, 34, 53),
  'algodesc': '\xe9\x87\x8d\xe6\x9e\x84\xe6\xb5\x8b\xe8\xaf\x95',
  'algoname': 'new_publish_mokuai',
  'atype': 'REST',
  'field': 2L,
  'funcslist': 'fib.fib',
  'host': '0.0.0.0',
  'insert_tm': datetime.datetime(2019, 1, 28, 18, 25, 26),
  'is_email': '\xe6\x98\xaf',
  'opertype': 'publish',
  'port': 48137L,
  'pyfile': 'arthur_runs/leepand6',
  'remark': '{n:10}',
  'status': 'normal',
  'tags': '\xe6\x9c\xba\xe5\x99\xa8\xe5\xad\xa6\xe4\xb9\xa0',
  'token': '20bf7679146eaef99136cde84ccc1eba',
  'uid': 1L,
  'version': 'v1.0.0'}]

In [9]:
# -*- coding: UTF-8 -*-
# author: leepand
# time: 2019-01-08
# desc: Algo 数据展示(my/other/zoo)

#update 2019-01-25

import datetime
from operator import itemgetter 
from Arthur.core.entities.bean.Arthur_service import ArthurService
from Arthur.core.entities.dao.Arthur_AlgoStar_dao import ArthurAlgoStarDao
#from Arthut.core.entities.dao.Arthur_serviceTags_dao import ArthurServTagsDao

class ArthurAlgoList:

    @staticmethod
    def _get_algolist_byuid(uId):
        #增加收藏的算法服务
        bRet,uid_algouid_list = ArthurAlgoStarDao.query_algouid_by_uidstar(uId)
        my_star_algo_list_all=[]
        my_star_algo_list=[]
        my_star_algo_list_dict=None
        if len(uid_algouid_list)>0:
            for algouid_algoname_dict in uid_algouid_list:
                algouid=algouid_algoname_dict['algouid']
                algoname = algouid_algoname_dict['algoname']
                #过滤掉自己收藏自己的算法，避免重复
                if int(algouid)-int(uId)==0:
                    continue
                bRet, my_star_algo_list_dict = ArthurService.service_user_list_filter(algouid,algoname,page=1,length=1000, status='normal', search=None)
                #print 'my_star_algo_list_dict',my_star_algo_list_dict
                if not bRet: my_star_algo_list = []
                if my_star_algo_list_dict is not None and "service_list" in my_star_algo_list_dict:
                    my_star_algo_list= my_star_algo_list_dict['service_list']
                else:
                    my_star_algo_list=[]
                my_star_algo_list_all.extend(my_star_algo_list)
    
        bRet, my_algo_list_dict = ArthurService.service_user_list(uId,page=1,length=1000, status='normal', search=None)
        if not bRet: my_algo_list = []
        if my_algo_list_dict is not None and "service_list" in my_algo_list_dict:
            my_algo_list= my_algo_list_dict['service_list']
        else:
            my_algo_list=[]
        my_algo_list.extend(my_star_algo_list_all)
        my_algo_list_new=my_algo_list
        return  {
            "algo_list": my_algo_list_new
        }
    
    @staticmethod
    def _get_algolist_zoo():
        bRet, algo_list_zoo = ArthurService.service_zoo_list(page=1,length=1000, status='normal', search=None)
        if not bRet: algo_list_zoo = []
        if  algo_list_zoo is not None and "service_list" in algo_list_zoo:
            serviceList=algo_list_zoo['service_list']
            #serviceList_bystar = sorted(serviceList,key = itemgetter('star_cnt'),reverse = True)
            serviceList_bytime = sorted(serviceList,key = itemgetter('insert_tm'),reverse = True)

        else:
            serviceList=[]
            serviceList_bystar=[]
            serviceList_bytime=[]
        return {
                "algo_list": serviceList,#serviceList
                "algo_list_bytime":serviceList_bytime}
    @staticmethod
    def _get_tag_services(tag):
        bRet, tag_services_list = ArthurService.service_tag_list(search=tag)
        if not bRet: tag_services_list = []
        if  len(tag_services_list)>0:
            serviceList_bytime = sorted(tag_services_list,key = itemgetter('insert_tm'),reverse = True)

        else:
            tag_services_list=[]
            serviceList_bytime=[]
        return {
                "algo_list": tag_services_list,#serviceList
                "algo_list_bytime":serviceList_bytime}


    @staticmethod
    def data_show(uId,query_type,tag):
        if query_type=='zoo':
            datas = {
                "dt_algo_list":ArthurAlgoList._get_algolist_zoo()
            }
        elif query_type=='user':
            datas = {
                "dt_algo_list":ArthurAlgoList._get_algolist_byuid(uId)
            }
        else:
            datas = {
                "dt_algo_list":ArthurAlgoList._get_tag_services(tag)
            }
        return True, datas
    @staticmethod
    def algo_list_forautocomplete():
        bRet, sRet = ArthurAlgoList.data_show(1,'zoo')
        new_serv_list=[]
        try:
            for _inter in sRet['dt_algo_list']['algo_list']:
                new_serv_list.append(_inter['algoname'])
        except:
            pass
        return True,new_serv_list

if __name__ == "__main__":

    bRet, sRet = ArthurAlgoList.data_show(1,'tag','Machine Learning')

    print '>>>> ', sRet

    #print HjsIndex._get_expire_order()


[30/01/2019 23:09:17][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_exec_cmdstr(106): select * from tb_algo where 1=1 and status = %s and locate(%s,tags)	['normal', 'Machine Learning']
[30/01/2019 23:09:18][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)
[30/01/2019 23:09:18][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_exec_cmdstr(106): select * from tb_user where uid = %s	[1L]
[30/01/2019 23:09:18][DEBUG]/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-2.0.1.dev1-py2.7.egg/Arthur/core/entities/base/bs_database_pid.py::_new_conn(31): conn(host: 127.0.0.1, port: 3306, user: root, passwd: , db: Arthur_manage)
[30/01/2019 23:09:18]

>>>>  {'dt_algo_list': {'algo_list_bytime': [<Storage {'algoname': 'None', 'status': 'normal', 'remark': '\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'insert_tm': '2019-01-29 08:27:11', 'uid': 1, 'atype': 'REST', 'token': '20bf7679146eaef99136cde84ccc1eba', 'field': 1L, 'algodesc': '\xe6\x9a\x82\xe6\x97\xa0\xe6\x8f\x8f\xe8\xbf\xb0\xef\xbc\x8c\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'host': '0.0.0.0', 'aid': 35, 'algo_tm': '2019-01-29', 'user_name': 'leepand6', 'port': 48322L}>, <Storage {'algoname': 'leepand13', 'status': 'normal', 'remark': '\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'insert_tm': '2019-01-28 14:51:57', 'uid': 2, 'atype': 'REST', 'token': '2a9f9670106e9ef9d456491840cc9eba', 'field': 1L, 'algodesc': '\xe6\x9a\x82\xe6\x97\xa0\xe6\x8f\x8f\xe8\xbf\xb0\xef\xbc\x8c\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\

In [10]:
sRet

{'dt_algo_list': {'algo_list': [<Storage {'algoname': 'leepand10', 'status': 'normal', 'remark': '\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'insert_tm': '2019-01-28 12:24:40', 'uid': 1, 'atype': 'REST', 'token': '20bf7679146eaef99136cde84ccc1eba', 'field': 1L, 'algodesc': '\xe6\x9a\x82\xe6\x97\xa0\xe6\x8f\x8f\xe8\xbf\xb0\xef\xbc\x8c\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'host': '0.0.0.0', 'aid': 31, 'algo_tm': '2019-01-28', 'user_name': 'leepand6', 'port': 22169L}>,
   <Storage {'algoname': 'leepand13', 'status': 'normal', 'remark': '\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\xe7\xbc\x96\xe8\xbe\x91\xe6\xb7\xbb\xe5\x8a\xa0', 'insert_tm': '2019-01-28 14:34:57', 'uid': 1, 'atype': 'REST', 'token': '20bf7679146eaef99136cde84ccc1eba', 'field': 1L, 'algodesc': '\xe6\x9a\x82\xe6\x97\xa0\xe6\x8f\x8f\xe8\xbf\xb0\xef\xbc\x8c\xe8\xaf\xb7\xe5\x88\xb0\xe4\xb8\xbb\xe9\xa1\xb5\x

In [4]:
from Arthur.core.entities.dao.Arthur_AlgoStar_dao import ArthurAlgoStarDao